In [1]:
import boto3

import os

import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

%load_ext autoreload
%autoreload 2

In [2]:
pwd

u'/Users/travis/Documents/Education/Galvanize/Capstone/src'

In [3]:
floc = '/Users/travis/Documents/Education/Galvanize/Capstone/data/yelpscraperesults/bizids-sorted-uniq.txt'
jfolder = '/Users/travis/Documents/Education/Galvanize/Capstone/Data/yelp-json-portland/'

files = os.listdir(jfolder)
files = set(files)

In [18]:
### only run this code if you need to update from s3 bucket

files = set(os.listdir(jfolder))

import sys
reload(sys)  # Reload does the trick!
sys.setdefaultencoding('UTF8')



bucket_name = 'wanderwell-world-portland'
fnames = []

s3 =  boto3.resource('s3')
s3bucket = s3.Bucket(bucket_name)
for name in s3bucket.objects.all():
    fnames.append(name.key)

len(fnames)

for name in s3bucket.objects.all():
    fname = name.key
    if fname not in files and '.json' in fname :
        print fname
        s3.meta.client.download_file(bucket_name,
                                     fname,
                                    '{}{}'.format(jfolder,fname))

### FLATTENING THE DATAFRAME

In [4]:
len(os.listdir(jfolder))

2989

In [9]:
daylist = ['mon', 'tues', 'wed', 'thurs', 'fri', 'sat', 'sun']
bad_list= []
poi_list = []
cat_set = set()
for bus in os.listdir(jfolder):
    if bus.split('.')[-1] == 'json':
            with open(jfolder+bus) as busfile:
                try: 
                    bus_json = json.load(busfile)
                except Exception:
                    print bus
                    bad_list.append(bus)
                    continue
                cats = bus_json.pop('categories')
                ### seperate hours into daily columns
                if 'hours' in bus_json:
                    hours_dict = bus_json.pop('hours')[0]['open']
                else:
                    hours_dict = None
                for idx, day in enumerate(daylist) :
                    try : 
                        bus_json['hours_open_{}'.format(day)] = hours_dict[idx]['start']
                    except Exception: 
                        bus_json['hours_open_{}'.format(day)] = False
                    try : 
                        bus_json['hours_closed_{}'.format(day)] = hours_dict[idx]['end']
                    except Exception: 
                        bus_json['hours_closed_{}'.format(day)] = False
                ### seprate catagories into individual columns (3 total)
                for idx, val in enumerate(cats):
                    bus_json['category-{}'.format(idx)] = val['alias']
                    cat_set.add('cat: {}'.format(val['alias']))
                poi_list.append(pd.io.json.json_normalize(bus_json))
            continue

yelp_df = pd.concat(poi_list)

for cat in ['category-0','category-1','category-2'] :
    yelp_df[cat].fillna(False,inplace=True)

hawthorne-hideaway-portland.json
lucky-horseshoe-lounge-portland.json
vista-spring-cafe-portland.json


In [10]:
yelp_df.columns

Index([              u'category-0',               u'category-1',
                     u'category-2',               u'category-3',
           u'coordinates.latitude',    u'coordinates.longitude',
                  u'display_phone',         u'hours_closed_fri',
               u'hours_closed_mon',         u'hours_closed_sat',
               u'hours_closed_sun',       u'hours_closed_thurs',
              u'hours_closed_tues',         u'hours_closed_wed',
                 u'hours_open_fri',           u'hours_open_mon',
                 u'hours_open_sat',           u'hours_open_sun',
               u'hours_open_thurs',          u'hours_open_tues',
                 u'hours_open_wed',                       u'id',
                      u'image_url',               u'is_claimed',
                      u'is_closed',        u'location.address1',
              u'location.address2',        u'location.address3',
                  u'location.city',         u'location.country',
         u'location.cross

In [16]:
yelp_df_ = yelp_df.set_index('id')
yelp_df_['review_count'].sum()

281757

In [11]:
yelp_df_['review_count'].nlargest(30)

id
voodoo-doughnut-portland                        6720
screen-door-portland                            4181
mothers-bistro-and-bar-portland                 2826
salt-and-straw-portland-2                       2402
pine-state-biscuits-portland-2                  2151
tasty-n-sons-portland                           2108
tasty-n-alder-portland                          1894
portland-city-grill-portland                    1890
lúc-lác-portland-3                              1503
voodoo-doughnut-too-portland                    1450
the-waffle-window-portland                      1354
toro-bravo-portland                             1198
tin-shed-portland                               1192
salt-and-straw-portland                         1187
le-bistro-montage-portland                      1078
jakes-famous-crawfish-portland                  1072
pambiche-portland                               1061
podnahs-pit-barbeque-portland                   1051
pips-original-doughnuts-portland-2         

In [12]:
yelp_df_['review_count'].mean()

94.35934360348293

In [17]:
n_photos = yelp_df_['photos'].apply(lambda x : len(x))
n_photos.nlargest(30)
n_photos.sum()

198937

In [11]:
yelp_df['category-0'].value_counts().nlargest(30)

coffee              381
pizza               142
newamerican         131
mexican             118
grocery             114
bakeries            113
vietnamese          106
hair                105
bars                102
sandwiches           96
thai                 87
parks                85
foodtrucks           85
autorepair           81
pubs                 73
generaldentistry     73
japanese             71
delis                66
italian              65
tradamerican         64
burgers              62
hotdogs              62
cafes                60
seafood              59
chinese              56
sushi                51
othersalons          50
realestateagents     50
lounges              47
convenience          47
Name: category-0, dtype: int64

### JSON FILE OF LIST OF CATAGORIES IS  defined below

In [12]:
catfile = '/Users/travis/Documents/Education/Galvanize/Capstone/Data/categories.json'

with open(catfile) as cats:
    cat_son = json.load(cats)

cat_df = pd.DataFrame(cat_son)

cat_key = cat_df.set_index('alias')['parents']
cat_key_m = cat_key.apply(lambda x: pd.Series(x))
cat_key_m = cat_key_m[0]
cat_key_m.head()

alias
3dprinting      localservices
abruzzese             italian
absinthebars             bars
acaibowls                food
accessories           fashion
Name: 0, dtype: object

In [13]:
cat_df.head()

,alias,country_blacklist,country_whitelist,parents,title
0,3dprinting,NaN,NaN,[localservices],3D Printing
1,abruzzese,NaN,[IT],[italian],Abruzzese
2,absinthebars,NaN,[CZ],[bars],Absinthe Bars
3,acaibowls,"[IT, MX, PL, AR, TR, CL]",NaN,[food],Acai Bowls
4,accessories,NaN,NaN,[fashion],Accessories


#### EXTRACT ALL THROW AWAY CATAGORIES

In [14]:
cat_df.head()
# cat_df.columns
cat_key_m.value_counts().shape

(104,)

In [15]:
cats_df = cat_key_m.reset_index()
cats_df.columns = ['catagory', 'parent']
cats_df.dropna(inplace=True)

has_service = cats_df['parent'].str.contains('service')
has_shopping = cats_df['parent'].str.contains('shopping')
has_health = cats_df['parent'].str.contains('health')
has_edu = cats_df['parent'].str.contains('education')
has_fit = cats_df['parent'].str.contains('fitness')
has_auto = cats_df['parent'].str.contains('auto')

dumps = has_service + has_health  + has_auto + has_edu + has_fit

dump_cats = cats_df[dumps.values]
has_dump = set(dump_cats['catagory'].values)

print dump_cats.shape
dump_cats.head()

,catagory,parent
0,3dprinting,localservices
8,acupuncture,health
10,adoptionservices,localservices
12,adultedu,education
15,aerialfitness,fitness


#### EXTRACT ALL FOOD CATAGORIES

In [16]:
cats_df = cat_key_m.reset_index()
cats_df.columns = ['catagory', 'parent']
food_bool = cats_df['parent'] == 'food'
rest_bool = cats_df['parent'] == 'restaurants'
all_food_bool = food_bool + rest_bool

food_cats = cats_df[all_food_bool]
has_food = set(food_cats['catagory'].values)

print food_cats.shape
food_cats.head()

,catagory,parent
3,acaibowls,food
17,afghani,restaurants
18,african,restaurants
37,andalusian,restaurants
49,arabian,restaurants


#### EXTRACT ALL NIGHTLIFE CATAGORIES

In [17]:
cats_df = cat_key_m.reset_index()
cats_df.columns = ['catagory', 'parent']
club_bool = cats_df['parent'] == 'nightlife'
bar_bool = cats_df['parent'] == 'bars'
beer_bool = cats_df['parent'].str.contains('beer').fillna(False)
wine_bool = cats_df['parent'].str.contains('wine').fillna(False)
cocktail_bool = cats_df['parent'].str.contains('cocktail').fillna(False)


all_drink_bool = club_bool + bar_bool + beer_bool + wine_bool + cocktail_bool

nightlife_cats = cats_df[all_drink_bool]
has_nightlife = set(nightlife_cats['catagory'].values)

print nightlife_cats.shape
nightlife_cats.head()

,catagory,parent
2,absinthebars,bars
13,adultentertainment,nightlife
26,airportlounges,bars
108,barcrawl,nightlife
110,bars,nightlife


In [18]:
len(has_nightlife)

40

#### EXTRACT ALL COFFEE CATAGORIES

In [19]:
cats_df = cat_key_m.reset_index()
cats_df.columns = ['catagory', 'parent']

has_coffee = cats_df['catagory'].str.contains('coffee').fillna(False)

coffee_cats = cats_df[has_coffee]
has_coffee = set(coffee_cats['catagory'].values)

print coffee_cats.shape
coffee_cats[coffee_cats['catagory'].str.contains('tea')].head()

,catagory,parent
286,coffeeteasupplies,food


## ADD CATAGORIES TO DATAFRAME

In [20]:
yelp_df_ = yelp_df.copy()

yelp_df_['coffee'] = yelp_df_['category-0'].isin(has_coffee)
yelp_df_['coffee'] = yelp_df_['coffee'] + yelp_df_['category-1'].isin(has_coffee)
yelp_df_['coffee'] = yelp_df_['coffee'] + yelp_df_['category-2'].isin(has_coffee)


yelp_df_['food'] = yelp_df_['category-0'].isin(has_food)
yelp_df_['food'] = yelp_df_['food'] + yelp_df_['category-1'].isin(has_food)
yelp_df_['food'] = yelp_df_['food'] + yelp_df_['category-2'].isin(has_food)

yelp_df_['nightlife'] = yelp_df_['category-0'].isin(has_nightlife)
yelp_df_['nightlife'] = yelp_df_['nightlife'] + yelp_df_['category-1'].isin(has_food)
yelp_df_['nightlife'] = yelp_df_['nightlife'] + yelp_df_['category-2'].isin(has_food)

yelp_df_['dump'] = yelp_df_['category-0'].isin(has_dump)
yelp_df_['dump'] = yelp_df_['nightlife'] + yelp_df_['category-1'].isin(has_dump)
yelp_df_['dump'] = yelp_df_['nightlife'] + yelp_df_['category-2'].isin(has_dump)
yelp_df_['dump'] = yelp_df_['dump'] * np.invert(yelp_df_['food'])
yelp_df_['dump'] = yelp_df_['dump'] * np.invert(yelp_df_['coffee'])
yelp_df_['dump'] = yelp_df_['dump'] * np.invert(yelp_df_['nightlife'])

In [21]:
yelp_df_ = yelp_df_[yelp_df_['dump'] ==  False]

In [22]:
yelp_df_.shape

(6240, 47)

In [23]:
yelp_df_.dump.sum()

0

In [24]:
yelp_df_.food.sum()

3383

In [25]:
yelp_df_.coffee.sum()

485

In [26]:
yelp_df_.nightlife.sum()

2400

In [27]:
cat_key_m['florists']

u'flowers'

In [28]:
cat_df[cat_df['alias'] == 'florists']

,alias,country_blacklist,country_whitelist,parents,title
491,florists,NaN,NaN,[flowers],Florists


### Plot datapoints on google map file

In [34]:
from gmplot import GoogleMapPlotter

s_lat = '47.629458' 
s_long = '-122.336338'

lats = yelp_df['coordinates.latitude'].values
longs = yelp_df['coordinates.longitude'].values


gmap = GoogleMapPlotter(s_lat, s_long, 13)

# gmap.plot(lats, longs, 'cornflowerblue', edge_width=10)
gmap.scatter(lats, longs, '#3B0B39', size=4, marker=False)
# gmap.scatter(lats, longs, 'k', marker=True)
# gmap.heatmap(lats, longs)

gmap.draw("../data/mymap_yelp.html")

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [ ]:
pwd

In [ ]:
for reviews, name in (yelp_df[['reviews', 'name']].values[:2,:]):
    print name

In [39]:
yelp_df_.columns

Index([              u'category-0',               u'category-1',
                     u'category-2',               u'category-3',
           u'coordinates.latitude',    u'coordinates.longitude',
                  u'display_phone',         u'hours_closed_fri',
               u'hours_closed_mon',         u'hours_closed_sat',
               u'hours_closed_sun',       u'hours_closed_thurs',
              u'hours_closed_tues',         u'hours_closed_wed',
                 u'hours_open_fri',           u'hours_open_mon',
                 u'hours_open_sat',           u'hours_open_sun',
               u'hours_open_thurs',          u'hours_open_tues',
                 u'hours_open_wed',                       u'id',
                      u'image_url',               u'is_claimed',
                      u'is_closed',        u'location.address1',
              u'location.address2',        u'location.address3',
                  u'location.city',         u'location.country',
         u'location.cross

In [42]:
coffee_df = yelp_df[yelp_df['category-0'].isin(has_coffee)]
coffee_df.columns

Index([              u'category-0',               u'category-1',
                     u'category-2',               u'category-3',
           u'coordinates.latitude',    u'coordinates.longitude',
                  u'display_phone',         u'hours_closed_fri',
               u'hours_closed_mon',         u'hours_closed_sat',
               u'hours_closed_sun',       u'hours_closed_thurs',
              u'hours_closed_tues',         u'hours_closed_wed',
                 u'hours_open_fri',           u'hours_open_mon',
                 u'hours_open_sat',           u'hours_open_sun',
               u'hours_open_thurs',          u'hours_open_tues',
                 u'hours_open_wed',                       u'id',
                      u'image_url',               u'is_claimed',
                      u'is_closed',        u'location.address1',
              u'location.address2',        u'location.address3',
                  u'location.city',         u'location.country',
         u'location.cross

In [44]:
from sklearn.cluster import KMeans

In [52]:
map_location = coffee_df[['coordinates.longitude', 'coordinates.latitude']]
# map_location = map_location[map_location['coordinates.longitude'] > 47.515]
# map_location = map_location[map_location['coordinates.longitude'] < 47.69468]
# map_location = map_location[map_location['coordinates.latitude'] < -122.420]
# map_location = map_location[map_location['coordinates.latitude'] > -122.222]



coffee_cluster = KMeans(n_clusters=9)

coffee_cluster.fit(map_location)

what_is_this = coffee_cluster.fit_predict(coffee_df[['coordinates.longitude', 'coordinates.latitude']])
clustered_lats = []
clustered_longs = []
for n, i in enumerate(what_is_this):
    lats = coffee_df[what_is_this == n]['coordinates.latitude'].values
    longs = coffee_df[what_is_this == n]['coordinates.longitude'].values
    clustered_lats.append(lats)
    clustered_longs.append(longs)

colors_list = ['crimson', 'darkseagreen', 
               'firebrick', 'dodgerblue', 
               'burlywood' 'gold',
               'indigo',  'navy', 
               'indianred', 'lawngreen']

longs = coffee_df['coordinates.longitude'].values
lats = coffee_df['coordinates.latitude'].values

s_lat = '47.629458' 
s_long = '-122.336338'

cent_long = coffee_cluster.cluster_centers_[:,0]
cent_lat = coffee_cluster.cluster_centers_[:,1]

gmap = GoogleMapPlotter(s_lat, s_long, 11.8)

for color, lats, longs in zip(colors_list, clustered_lats, clustered_longs):
    gmap.scatter(lats, longs, color, size=50, marker=False)

# gmap.plot(lats, longs, 'cornflowerblue', edge_width=10)
# gmap.scatter(lats, longs, '#3B0B39', size=40, marker=False)
# gmap.scatter(lats, longs, 'k', marker=True)
# gmap.heatmap(lats, longs)
c_longs = []
c_lats = []
for line in coffee_cluster.cluster_centers_:
    c_longs.append(line[0])
    c_lats.append(line[1])
gmap.scatter(c_lats, c_longs, 'hotpink', size=40, marker=True)

gmap.draw("../plots/coffee_centers.html")